In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torch-geometric pandas numpy scikit-learn networkx matplotlib seaborn tqdm -q

In [1]:
import time

import numpy as np
import pandas as pd
import psutil
import torch
import torch.nn.functional as F
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv, GCNConv, SAGEConv
from torch_geometric.utils import to_undirected
from ucimlrepo import fetch_ucirepo

try:
    import pynvml
    pynvml.nvmlInit()
    NVML_AVAILABLE = True
except:
    NVML_AVAILABLE = False

/home/klema/miniconda3/envs/graphsage/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CSV_PATH = '/home/klema/sibnn/gnn_tbank/check_notebooks/data/accepted_2007_to_2018Q4.csv'

Lending club dataset (2007-2018)
При загрузке данных отбираем только релевантные колонки:
1. Числовые: сумма кредита, ставка, доход и др
2. Категориальные: Цель кредита, рейтинг, стаж и др

Целевая переменная - loan_status

In [3]:
print("Загрузка данных")

usecols = [
    'loan_amnt', 'int_rate', 'installment', 'grade', 'emp_length', 
    'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
    'purpose', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc',
    'pub_rec', 'revol_bal', 'revol_util', 'total_acc'
]

df = pd.read_csv(CSV_PATH, usecols=usecols, low_memory=False)

# Фильтруем только завершенные займы. Fully Paid - успешно погашен, Charged Off - дефолт.
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]
df['target'] = df['loan_status'].map({'Fully Paid': 0, 'Charged Off': 1})

numeric_features = ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
                   'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 
                   'revol_bal', 'revol_util', 'total_acc']
categorical_features = ['grade', 'emp_length', 'home_ownership', 'verification_status', 'purpose']

X_num = df[numeric_features].copy()
X_num.replace([np.inf, -np.inf], np.nan, inplace=True)
X_num.fillna(X_num.median(), inplace=True)

X_cat = df[categorical_features].copy()
X_cat.fillna('Unknown', inplace=True)
X_cat_dummies = pd.get_dummies(X_cat, drop_first=True)

X = pd.concat([X_num, X_cat_dummies], axis=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = df['target'].values

# Строим разреженный граф симметричных связей на основе признакового сходства методом k-ближайших соседей.
k = 5
adj_matrix = kneighbors_graph(X_scaled, k, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
edge_index = to_undirected(edge_index)

num_nodes = len(X_scaled)
indices = np.arange(num_nodes)
train_idx, temp_idx = train_test_split(indices, test_size=0.4, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

data = Data(
    x=torch.tensor(X_scaled, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(y, dtype=torch.long),
    train_mask=train_mask,
    val_mask=val_mask,
    test_mask=test_mask
)

print(f"Загружено {data.x.shape[0]} узлов, {data.x.shape[1]} признаков")
print(f"Train/Val/Test: {train_mask.sum().item()}/{val_mask.sum().item()}/{test_mask.sum().item()}")
print(f"Рёбер: {edge_index.shape[1]}")

class_counts = torch.bincount(data.y).tolist()
class_dist_str = ", ".join(f"Класс {i}: {count}" for i, count in enumerate(class_counts))
print(f"Распределение классов: {class_dist_str}")

Загрузка данных
Загружено 1345310 узлов, 49 признаков
Train/Val/Test: 807186/269062/269062
Рёбер: 9756360
Распределение классов: Класс 0: 1076751, Класс 1: 268559


In [4]:
# Вычисление весов перед обучением
y_train = data.y[data.train_mask].cpu().numpy()
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(data.x.device)

# Использование в лоссе
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

In [5]:
def compute_metrics(pred, true, probs=None):
    pred = pred.cpu().numpy()
    true = true.cpu().numpy()
    
    metrics = {
        'accuracy': (pred == true).mean(),
        'precision': precision_score(true, pred, zero_division=0),
        'recall': recall_score(true, pred, zero_division=0),
        'f1': f1_score(true, pred, zero_division=0),
    }
    
    if probs is not None:
        probs = probs.cpu().numpy()
        metrics['roc_auc'] = roc_auc_score(true, probs)
        metrics['pr_auc'] = average_precision_score(true, probs)
    
    return metrics

def accuracy(pred_y, y):
    return ((pred_y == y).sum() / len(y)).item()

def test(model, data):
    """Тестирование с полным набором метрик"""
    model.eval()
    with torch.no_grad():
        _, out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)[data.test_mask]
        probs = torch.exp(out)[:, 1][data.test_mask]  # Вероятность класса 1 (дефолт)
        true = data.y[data.test_mask]
        
        return compute_metrics(pred, true, probs)

In [6]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.sage1 = SAGEConv(dim_in, dim_h)
        self.sage2 = SAGEConv(dim_h, dim_out)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                        lr=0.01,
                                        weight_decay=5e-4)

    def forward(self, x, edge_index):
        h = self.sage1(x, edge_index)
        h = torch.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.sage2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        # Взвешенный лосс
        y_train = data.y[data.train_mask].cpu().numpy()
        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        weights = torch.tensor(weights, dtype=torch.float).to(data.x.device)
        criterion = torch.nn.CrossEntropyLoss(weight=weights)
        optimizer = self.optimizer

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            _, out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                train_pred = out[data.train_mask].argmax(dim=1)
                train_metrics = compute_metrics(train_pred, data.y[data.train_mask])
                
                val_pred = out[data.val_mask].argmax(dim=1)
                val_probs = torch.exp(out)[:, 1][data.val_mask]
                val_metrics = compute_metrics(val_pred, data.y[data.val_mask], probs=val_probs)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            if epoch % 10 == 0:
                print(f'Epoch {epoch:>3} | '
                    f'TL: {loss:.3f} | TF1: {train_metrics["f1"]:.3f} | '
                    f'VL: {val_loss:.3f} | VF1: {val_metrics["f1"]:.3f} | '
                    f'VRec: {val_metrics["recall"]:.3f} | VPR: {val_metrics["pr_auc"]:.3f}')

class GAT(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out, heads=4):
        super().__init__()
        self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads, concat=True, dropout=0.6)
        self.gat2 = GATv2Conv(dim_h * heads, dim_out, heads=heads, concat=False, dropout=0.6)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                        lr=0.005,
                                        weight_decay=5e-4)

    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.6, training=self.training)
        h = self.gat1(h, edge_index)
        h = F.elu(h)
        h = F.dropout(h, p=0.6, training=self.training)
        h = self.gat2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        # Взвешенный лосс
        y_train = data.y[data.train_mask].cpu().numpy()
        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        weights = torch.tensor(weights, dtype=torch.float).to(data.x.device)
        criterion = torch.nn.CrossEntropyLoss(weight=weights)
        optimizer = self.optimizer

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            _, out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                train_pred = out[data.train_mask].argmax(dim=1)
                train_metrics = compute_metrics(train_pred, data.y[data.train_mask])
                
                val_pred = out[data.val_mask].argmax(dim=1)
                val_probs = torch.exp(out)[:, 1][data.val_mask]
                val_metrics = compute_metrics(val_pred, data.y[data.val_mask], probs=val_probs)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            if epoch % 10 == 0:
                print(f'Epoch {epoch:>3} | '
                    f'TL: {loss:.3f} | TF1: {train_metrics["f1"]:.3f} | '
                    f'VL: {val_loss:.3f} | VF1: {val_metrics["f1"]:.3f} | '
                    f'VRec: {val_metrics["recall"]:.3f} | VPR: {val_metrics["pr_auc"]:.3f}')

class GCN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_out)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                        lr=0.01,
                                        weight_decay=5e-4)

    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.5, training=self.training)
        h = self.gcn1(h, edge_index)
        h = torch.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.gcn2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        # Взвешенный лосс
        y_train = data.y[data.train_mask].cpu().numpy()
        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        weights = torch.tensor(weights, dtype=torch.float).to(data.x.device)
        criterion = torch.nn.CrossEntropyLoss(weight=weights)
        optimizer = self.optimizer

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            _, out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                train_pred = out[data.train_mask].argmax(dim=1)
                train_metrics = compute_metrics(train_pred, data.y[data.train_mask])
                
                val_pred = out[data.val_mask].argmax(dim=1)
                val_probs = torch.exp(out)[:, 1][data.val_mask]
                val_metrics = compute_metrics(val_pred, data.y[data.val_mask], probs=val_probs)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            if epoch % 10 == 0:
                print(f'Epoch {epoch:>3} | '
                    f'TL: {loss:.3f} | TF1: {train_metrics["f1"]:.3f} | '
                    f'VL: {val_loss:.3f} | VF1: {val_metrics["f1"]:.3f} | '
                    f'VRec: {val_metrics["recall"]:.3f} | VPR: {val_metrics["pr_auc"]:.3f}')

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

def monitor_resources():
    stats = {}
    # CPU & RAM
    stats['ram_mb'] = psutil.virtual_memory().used / (1024 ** 2)
    stats['cpu_percent'] = psutil.cpu_percent()

    # GPU
    if device.type == 'cuda' and NVML_AVAILABLE:
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        util = pynvml.nvmlDeviceGetUtilizationRates(handle)
        stats['gpu_mem_mb'] = mem_info.used / (1024 ** 2)
        stats['gpu_util'] = util.gpu
    else:
        stats['gpu_mem_mb'] = None
        stats['gpu_util'] = None
    return stats

def train_with_monitoring(model, data, epochs, model_name):
    print(f"\n{'='*50}\nTraining {model_name} with resource monitoring\n{'='*50}")
    
    if device.type == 'cuda':
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()

    start_time = time.time()

    model.fit(data, epochs)

    final_ram = psutil.virtual_memory().used / (1024 ** 2)
    max_gpu_mem = None
    if device.type == 'cuda':
        max_gpu_mem = torch.cuda.max_memory_allocated() / (1024 ** 2)

    duration = time.time() - start_time
    test_metrics = test(model, data)

    results = {
        'test_metrics': test_metrics,
        'training_time_sec': duration,
        'final_ram_mb': final_ram,
        'max_gpu_mem_mb': max_gpu_mem,
    }

    print(f"\n{model_name} finished")
    print(f"Test Recall: {test_metrics.get('recall', 0):.3f} | "
          f"F1: {test_metrics.get('f1', 0):.3f} | "
          f"PR-AUC: {test_metrics.get('pr_auc', 0):.3f}")
    print(f"Training Time: {duration:.1f} sec")
    if max_gpu_mem:
        print(f"Peak GPU Memory: {max_gpu_mem:.1f} MB")
    print(f"Final RAM Usage: {final_ram:.1f} MB")

    return test_metrics.get('recall', 0), results

In [8]:
%%time

results = {}

# 1. GraphSAGE
graphsage = GraphSAGE(data.x.shape[1], dim_h=64, dim_out=2).to(device)
acc_sage, _ = train_with_monitoring(graphsage, data, epochs=100, model_name="GraphSAGE")
results['GraphSAGE'] = acc_sage


Training GraphSAGE with resource monitoring
Epoch   0 | TL: 0.783 | TF1: 0.219 | VL: 0.782 | VF1: 0.219 | VRec: 0.273 | VPR: 0.187
Epoch  10 | TL: 0.645 | TF1: 0.408 | VL: 0.644 | VF1: 0.406 | VRec: 0.732 | VPR: 0.329
Epoch  20 | TL: 0.631 | TF1: 0.420 | VL: 0.630 | VF1: 0.419 | VRec: 0.678 | VPR: 0.352
Epoch  30 | TL: 0.628 | TF1: 0.421 | VL: 0.627 | VF1: 0.421 | VRec: 0.689 | VPR: 0.357
Epoch  40 | TL: 0.627 | TF1: 0.423 | VL: 0.626 | VF1: 0.422 | VRec: 0.686 | VPR: 0.359
Epoch  50 | TL: 0.625 | TF1: 0.424 | VL: 0.624 | VF1: 0.423 | VRec: 0.679 | VPR: 0.362
Epoch  60 | TL: 0.625 | TF1: 0.425 | VL: 0.624 | VF1: 0.423 | VRec: 0.676 | VPR: 0.363
Epoch  70 | TL: 0.624 | TF1: 0.426 | VL: 0.623 | VF1: 0.425 | VRec: 0.682 | VPR: 0.366
Epoch  80 | TL: 0.623 | TF1: 0.426 | VL: 0.623 | VF1: 0.426 | VRec: 0.681 | VPR: 0.366
Epoch  90 | TL: 0.623 | TF1: 0.427 | VL: 0.622 | VF1: 0.426 | VRec: 0.680 | VPR: 0.368
Epoch 100 | TL: 0.622 | TF1: 0.427 | VL: 0.622 | VF1: 0.426 | VRec: 0.681 | VPR: 0.36

In [9]:
%%time

# 2. GCN
gcn = GCN(data.x.shape[1], dim_h=64, dim_out=2).to(device)
acc_gcn, _ = train_with_monitoring(gcn, data, epochs=100, model_name="GCN")
results['GCN'] = acc_gcn


Training GCN with resource monitoring
Epoch   0 | TL: 0.919 | TF1: 0.212 | VL: 0.921 | VF1: 0.212 | VRec: 0.297 | VPR: 0.173
Epoch  10 | TL: 0.666 | TF1: 0.405 | VL: 0.665 | VF1: 0.403 | VRec: 0.599 | VPR: 0.327
Epoch  20 | TL: 0.645 | TF1: 0.404 | VL: 0.644 | VF1: 0.403 | VRec: 0.624 | VPR: 0.328
Epoch  30 | TL: 0.636 | TF1: 0.415 | VL: 0.635 | VF1: 0.414 | VRec: 0.657 | VPR: 0.339
Epoch  40 | TL: 0.634 | TF1: 0.416 | VL: 0.633 | VF1: 0.415 | VRec: 0.681 | VPR: 0.343
Epoch  50 | TL: 0.632 | TF1: 0.417 | VL: 0.632 | VF1: 0.416 | VRec: 0.680 | VPR: 0.346
Epoch  60 | TL: 0.631 | TF1: 0.418 | VL: 0.631 | VF1: 0.417 | VRec: 0.679 | VPR: 0.347
Epoch  70 | TL: 0.631 | TF1: 0.419 | VL: 0.630 | VF1: 0.417 | VRec: 0.679 | VPR: 0.349
Epoch  80 | TL: 0.630 | TF1: 0.419 | VL: 0.630 | VF1: 0.418 | VRec: 0.681 | VPR: 0.350
Epoch  90 | TL: 0.630 | TF1: 0.420 | VL: 0.630 | VF1: 0.418 | VRec: 0.682 | VPR: 0.351
Epoch 100 | TL: 0.630 | TF1: 0.420 | VL: 0.629 | VF1: 0.419 | VRec: 0.685 | VPR: 0.351

GCN

In [ ]:
%%time

# 3. GAT
gat = GAT(data.x.shape[1], dim_h=32, dim_out=2, heads=4).to(device)
acc_gat, _ = train_with_monitoring(gat, data, epochs=100, model_name="GAT")
results['GAT'] = acc_gat